In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Imported API key
from config import g_key

In [64]:
# Loaded the csv exported in Part I to a DataFrame

csv_path = '../WeatherPy/Output/cities.csv'
cities_df = pd.read_csv(csv_path)
cities_df.head()

,City,Cloudiness (%),Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Saint Anthony,75,US,1584070990,64,45.02,-93.22,35.60,14.99
1,Am Timan,66,TD,1584070991,15,11.03,20.28,68.77,8.75
2,Vardø,100,NO,1584070992,86,70.37,31.11,23.56,30.69
3,Marawi,0,PH,1584070993,84,8.00,124.29,82.99,7.00
4,Lüderitz,0,NaN,1584070994,82,-26.65,15.16,64.40,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
# Configured gmaps
gmaps.configure(api_key=g_key)

cities_df.dtypes

City               object
Cloudiness (%)      int64
Country            object
Date                int64
Humidity            int64
Latitude          float64
Longitude         float64
Max Temp          float64
Wind Speed        float64
dtype: object

In [66]:
# Storing latitude and longitude in locations
locations = cities_df[['Latitude', 'Longitude']].astype(float)

# Setting humidity as weight
humid = cities_df['Humidity']

In [68]:
# Plotted Heatmap
fig = gmaps.Map()

# Created heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=cities_df['Humidity'].max(),point_radius=4,opacity=0.5)
# Adjusted Dimensions
figure_layout = {'width': '1000px',
                 'height': '800px',
                 'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,center=(0,0),zoom_level=2)

# Added layer
fig.add_layer(heat_layer)

# Displayed figure
fig

Figure(layout=FigureLayout(height='800px', margin='0 auto 0 auto', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [69]:
# I would like to go somewhere with weather conditions similar to Bora Bora:
# Temp: 73 F to 85 F
# Humidity : 55% to 75%
# Wind Speed : 12 - 14.5 mph

# Narrowing cities with similar conditions
vacation_df = cities_df.loc[(cities_df['Max Temp'] >= 73) & (cities_df['Max Temp'] <= 85) & (cities_df['Humidity'] >= 55) & (cities_df['Humidity'] <= 75) & (cities_df['Wind Speed'] >= 12) & (cities_df['Wind Speed'] <= 14.5)]

# Dropping empty rows, if any
vacation_df = vacation_df.dropna(how='any')
vacation_df

,City,Cloudiness (%),Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
51,Hithadhoo,1,MV,1584071055,69,-0.60,73.08,83.57,13.09
88,Hilo,1,US,1584070807,65,19.73,-155.09,77.00,12.75
357,Lorengau,32,PG,1584071425,72,-2.02,147.27,84.79,13.91
424,Vila do Maio,0,CV,1584071505,74,15.13,-23.22,78.80,12.75
439,Iralaya,0,HN,1584071522,71,15.00,-83.23,79.25,12.97
510,Batemans Bay,0,AU,1584071608,68,-35.72,150.18,75.99,13.00
538,Alice Town,37,BS,1584071643,71,25.72,-79.30,74.82,13.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
# Setting up hotel_df dataframw with additional columns to hold information
hotel_df = pd.DataFrame(vacation_df[['City','Country','Latitude','Longitude']],columns=['City','Country','Latitude','Longitude'])
hotel_df['Name of the Hotel'] = ""

hotel_df.head()

,City,Country,Latitude,Longitude,Name of the Hotel
51,Hithadhoo,MV,-0.60,73.08,
88,Hilo,US,19.73,-155.09,
357,Lorengau,PG,-2.02,147.27,
424,Vila do Maio,CV,15.13,-23.22,
439,Iralaya,HN,15.00,-83.23,


In [71]:
# Find the closest hotel within 5000m of each city, using 'lodging' type, as there is no type 'hotel' according to Google APIs documentation
# Doucumentation Link- https://developers.google.com/places/web-service/supported_types

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "lodging",
          "key": g_key,
          "radius": 5000
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # add latitude and longitude type from vacation df to params dict
    params["location"] = f"{row['Latitude']},{row['Longitude']}"
    # print(params)

    # assemble url and make API request
    print(f"Retrieving Hotels near for city: {row['City']}")
    response = requests.get(base_url, params=params).json()

    # extract results
    try:
        print(f"Nearest hotel in {row['City']}: {response['results'][0]['name']}.")     
        hotel_df.loc[index, 'Name of the Hotel'] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")  

Retrieving Hotels near for city: Hithadhoo
Nearest hotel in Hithadhoo: Scoop Guest House.
------------
Retrieving Hotels near for city: Hilo
Nearest hotel in Hilo: Hilo Hawaiian Hotel.
------------
Retrieving Hotels near for city: Lorengau
Nearest hotel in Lorengau: Lorengau Harbourside Hotel.
------------
Retrieving Hotels near for city: Vila do Maio
Nearest hotel in Vila do Maio: Residence 'I Delfini'.
------------
Retrieving Hotels near for city: Iralaya
Nearest hotel in Iralaya: Iglesia morava renovada.
------------
Retrieving Hotels near for city: Batemans Bay
Nearest hotel in Batemans Bay: Coachhouse Marina Resort - Batemans Bay.
------------
Retrieving Hotels near for city: Alice Town
Nearest hotel in Alice Town: Bimini Big Game Club Resort & Marina.
------------


In [72]:
# Displaying df to check if all hotel names have been stored correctly
hotel_df

,City,Country,Latitude,Longitude,Name of the Hotel
51,Hithadhoo,MV,-0.60,73.08,Scoop Guest House
88,Hilo,US,19.73,-155.09,Hilo Hawaiian Hotel
357,Lorengau,PG,-2.02,147.27,Lorengau Harbourside Hotel
424,Vila do Maio,CV,15.13,-23.22,Residence 'I Delfini'
439,Iralaya,HN,15.00,-83.23,Iglesia morava renovada
510,Batemans Bay,AU,-35.72,150.18,Coachhouse Marina Resort - Batemans Bay
538,Alice Town,BS,25.72,-79.30,Bimini Big Game Club Resort & Marina


In [73]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name of the Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [74]:
# Adding marker locations and info boxes
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='800px', margin='0 auto 0 auto', width='1000px'))